In [95]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

In [96]:
import xgboost

In [97]:
from xgboost import XGBClassifier

In [98]:
xgboost.__version__

'1.7.4'

In [99]:
data = pd.read_csv('../Field_training_data_noOutliers_newContaminationMetric.csv')
labels = pd.read_csv('../Field_training_labels_noOutliers_newContaminationMetric.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams_noOutliers_newContaminationMetric_xg.csv')

testTranscriptomes_TPM_merged_lats = pd.read_csv('../../ForElaina/testTranscriptomes_updated.csv')

In [100]:
testTranscriptomes_TPM_merged_lats

,PF00003,PF00013,PF00022,PF00034,PF00071,PF00124,PF00125,PF00139,PF00169,PF00185,...,PF14259,PF14727,PF14778,PF14782,PF14870,PF15104,PF15782,PF16158,PF16313,PF16672
0,10.046575,158.030067,25988.574992,112.935233,4499.331731,33.123315,1254.456191,8.453067,876.477907,23.890872,...,127.001966,1.508095,2.118179,9.159643,0.610866,96.769656,6.016799,124.205379,1.363049,13.681869
1,7.811929,157.474385,17880.008120,167.023186,4537.827236,9.917466,1614.436955,3.592029,1017.085206,20.106738,...,130.842684,4.038135,0.745770,8.229209,0.854796,65.681446,3.451634,101.944170,2.374882,12.277930
2,787.014850,195.224998,174.733693,0.000000,772.934705,46.709280,653.002416,2.803057,87.831549,106.497819,...,0.000000,0.000000,0.000000,0.000000,368.240547,0.000000,0.000000,0.000000,0.000000,0.000000
3,512.259396,248.241213,160.236309,0.000000,934.561313,45.043389,818.936266,5.603980,124.957392,198.381424,...,0.000000,0.000000,0.000000,0.000000,335.669983,0.000000,0.000000,0.000000,0.000000,0.000000
4,260.384332,343.268010,57.555141,0.000000,4203.319313,323.063582,801.212518,6.612931,65.526981,330.326658,...,0.000000,0.000000,0.000000,0.000000,280.710914,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,0.000000,691.131962,452.356021,158.707529,1068.512981,96.562902,118.398317,0.000000,72.394357,16.666724,...,132.591401,6.260450,2.540900,5.138098,97.311203,27.302130,0.000000,56.711331,3.097510,0.000000
131,0.000000,482.920475,352.272713,24.486383,856.962360,433.498823,108.945900,3.537033,60.176887,7.811652,...,77.027271,3.305202,2.474822,9.245085,56.001834,26.873542,0.000000,57.509667,0.000000,0.000000
132,0.000000,664.816686,348.040663,162.871551,1020.256387,240.017543,356.546399,15.707057,83.916954,19.919499,...,105.604299,6.105135,4.607427,10.396274,112.783560,32.281206,0.000000,79.329132,1.417856,0.000000
133,0.000000,1188.584821,534.427830,22.377412,1226.042678,61.994218,209.057832,1.098122,91.723780,26.207345,...,104.134344,3.602228,3.684826,11.233160,97.021471,33.805776,0.000000,87.050256,2.809811,0.000000


In [101]:
testTranscriptomes_TPM_merged_lats.columns[(testTranscriptomes_TPM_merged_lats == 0).all()]

Index([], dtype='object')

In [102]:
features = features['pfam']

In [103]:
len(features)

183

In [104]:
testTranscriptomes_TPM_merged_lats = testTranscriptomes_TPM_merged_lats[features]

In [105]:
le = LabelEncoder()

In [106]:
train_data = train_data[features]

In [107]:
from sklearn.model_selection import train_test_split
from sklearn import svm

In [108]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [109]:
#from https://scikit-learn.org/stable/modules/cross_validation.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [110]:
testTranscriptomes = scaler.transform(testTranscriptomes_TPM_merged_lats)

In [111]:
#{'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'reg_lambda': 1.0} 
model = XGBClassifier(gamma=0.0, learning_rate=0.1, max_depth=3, n_estimators=1000, reg_lambda=1.0)
model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [112]:
xg_predictions_testTranscriptomes = model.predict(testTranscriptomes)

In [113]:
prob = model.predict_proba(testTranscriptomes)

In [114]:
probDF = pd.DataFrame(data={'probability':np.max(prob, axis = 1)})

In [115]:
testTranscriptomes_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_testTranscriptomes})

In [116]:
testTranscriptomes_predictions

,xg_pred
0,2
1,2
2,2
3,2
4,2
...,...
130,2
131,2
132,1
133,1


In [117]:
testTranscriptomes_predictions.to_csv('../../ForElaina/testTranscriptomes_predictions',index=False)

In [118]:
probDF.to_csv('../../ForElaina/testTranscriptomes_probability',index=False)